# Final Project #
#### CSCI 4831 - Sabermetrics ####
#### Instructor: Rhonda Hoenigman ####
#### Name: Hussain Quadri ####

In [1]:
#Import Statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pybaseball import pitching_stats_range, pitching_stats_bref
from pybaseball import statcast
%matplotlib inline


## Data & Statistics

In [2]:
# Data grabbing
# data = statcast('2017-01-01', '2017-12-31')
pitchers = pd.read_csv("Statcast_Pitching_Data_2017.csv", low_memory=False)

In [3]:
pitchers.head(5)

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,SL,2017-10-01,84.8,-1.7864,6.2820,George Kontos,645302,502004,strikeout,swinging_strike,...,8,11,8,11,11,8,8,11,Standard,Standard
1,SL,2017-10-01,85.5,-1.6885,6.2841,George Kontos,645302,502004,NaN,blocked_ball,...,8,11,8,11,11,8,8,11,Standard,Standard
2,SL,2017-10-01,85.5,-1.7503,6.3259,George Kontos,645302,502004,NaN,swinging_strike,...,8,11,8,11,11,8,8,11,Standard,Standard
3,FC,2017-10-01,89.1,-2.0060,6.2080,George Kontos,645302,502004,NaN,called_strike,...,8,11,8,11,11,8,8,11,Standard,Standard
4,FT,2017-10-01,95.3,1.8362,6.1102,Dan Runzler,457477,502130,single,hit_into_play_score,...,7,11,7,11,11,7,7,11,Standard,Standard


In [4]:
# Filter data for just bb_type, v(x,y,z)0/a(x,y,z) effective_speed, release_spin, launch_speed, and launch_speed_angle

filtered_data = pitchers[['player_name',
                 'pitcher',
                 'bb_type',
                 'vx0', 'vy0', 'vz0',
                 'ax', 'ay', 'az',
                 'effective_speed',
                 'release_spin_rate',
                 'launch_speed',
                 'launch_speed_angle']]
filtered_data.groupby('player_name', as_index = False).count().head(10)

,player_name,pitcher,bb_type,vx0,vy0,vz0,ax,ay,az,effective_speed,release_spin_rate,launch_speed,launch_speed_angle
0,A.J. Cole,159,26,159,159,159,159,159,159,159,150,45,26
1,A.J. Griffin,45,13,44,44,44,44,44,44,44,44,17,13
2,A.J. Minter,85,9,85,85,85,85,85,85,85,84,22,9
3,A.J. Schugel,28,4,28,28,28,28,28,28,28,28,5,4
4,AJ Ramos,26,3,26,26,26,26,26,26,26,26,6,3
5,Aaron Bummer,62,8,62,62,62,62,62,62,62,61,13,8
6,Aaron Loup,54,6,54,54,54,54,54,54,54,54,13,6
7,Aaron Nola,106,14,106,106,106,106,106,106,106,106,20,14
8,Aaron Slegers,70,14,70,70,70,70,70,70,70,70,21,14
9,Aaron Wilkerson,86,18,85,85,85,85,85,85,85,85,25,18
